**Импортируем библиотеки**

In [ ]:
#импортируем библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer, LabelEncoder, MinMaxScaler, StandardScaler

In [ ]:
#подгружаем первый файл и смотрим размеpность
dataset = pd.read_excel(r'/content/dataset.xlsx')
dataset.shape

**Проведем предобработку данных**

In [ ]:
dataset = dataset.replace({'Угол нашивки, град': {0.0 : 0, 90.0 : 1}})
dataset = dataset.rename(columns={'Угол нашивки, град' : 'Угол нашивки'})
dataset['Угол нашивки'] = dataset['Угол нашивки'].astype(int)
dataset

In [ ]:
#создадим новый файл
dataset.to_excel('dataset1.xlsx', index = False)
print('Файл успешно сохранился!')

In [ ]:
#среднее значение
dataset.mean()

In [ ]:
#медианное значение
dataset.median()

**Удаление выбросов**

In [ ]:
dataset.isna().sum()

In [ ]:
dataset.describe()

**Удаляем выбросы**

In [ ]:
#для удаления выбросов используем метод межквартильных расстояний
metod_iq = 0
count_iq = [] #список, куда записывается количество выбросов по каждой переменной
for column in dataset:
    d = dataset.loc[:, [column]]
    q1 = np.quantile(dataset[column], 0.25)
    q3 = np.quantile(dataset[column], 0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    d['iq'] = (dataset[column] <= lower) | (dataset[column] >= upper)
    metod_iq += d['iq'].sum()
    count_iq.append(d['iq'].sum())
    print(column, ': ', d['iq'].sum())
print('Метод межквартильных расстояний, выбросов:', metod_iq)

In [ ]:
#cоздадим переменную со списком всех параметров, в которых есть выбросы за исклюением целевых переменных
dataset.columns
column_list_drop = ['Плотность, кг/м3',
                    'Модуль упругости, ГПа',
                    'Количество отвердителя, м.%',
                    'Содержание эпоксидных групп,%_2',
                    'Температура вспышки, С_2',
                    'Поверхностная плотность, г/м2',
                    'Потребление смолы, г/м2',
                    'Шаг нашивки',
                    'Плотность нашивки']

In [ ]:
#исключим выбросы, очистим данные от выбросов методом межквартильного расстояния (далее 1,5 межквартильных размахов)
for i in column_list_drop:
    q3, q1 = np.percentile(dataset.loc[:,i], [75,25])
    iqr = q3 - q1
    max = q3 + (1.5 * iqr)
    min = q1 - (1.5 * iqr)
    dataset.loc[dataset[i] < min, i] = np.nan
    dataset.loc[dataset[i] > max, i] = np.nan

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset = dataset.dropna(axis=0)

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.info()

In [ ]:
for i in column_list_drop:
    q3, q1 = np.percentile(dataset.loc[:,i], [75,25])
    iqr = q3 - q1
    max = q3 + (1.5 * iqr)
    min = q1 - (1.5 * iqr)
    dataset.loc[dataset[i] < min, i] = np.nan
    dataset.loc[dataset[i] > max, i] = np.nan

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset = dataset.dropna(axis=0)

In [ ]:
dataset.isnull().sum()

In [ ]:
for i in column_list_drop:
    q3, q1 = np.percentile(dataset.loc[:,i], [75,25])
    iqr = q3 - q1
    max = q3 + (1.5 * iqr)
    min = q1 - (1.5 * iqr)
    dataset.loc[dataset[i] < min, i] = np.nan
    dataset.loc[dataset[i] > max, i] = np.nan

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset = dataset.dropna(axis=0)

In [ ]:
dataset.isnull().sum()

In [ ]:
for i in column_list_drop:
    q3, q1 = np.percentile(dataset.loc[:,i], [75,25])
    iqr = q3 - q1
    max = q3 + (1.5 * iqr)
    min = q1 - (1.5 * iqr)
    dataset.loc[dataset[i] < min, i] = np.nan
    dataset.loc[dataset[i] > max, i] = np.nan

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.info()

Выбросы удалили за 3 подхода

**Мы избавились от шумов, выбросов, строим графики**

In [ ]:
#построим гистограммы распределения каждой из переменных
dataset.hist(figsize = (20,20), color = 'b')
plt.show()

In [ ]:
#попарные графики рассеяния точек
sns.set_style('darkgrid')
sns.pairplot(dataset,
            x_vars=['Соотношение матрица-наполнитель', 'Плотность, кг/м3',
                    'Модуль упругости, ГПа', 'Количество отвердителя, м.%',
                    'Содержание эпоксидных групп,%_2', 'Температура вспышки, С_2'],
             y_vars=['Модуль упругости при растяжении, ГПа'],
             palette='bright')

In [ ]:
#попарные графики рассеяния точек
sns.set_style('darkgrid')
sns.pairplot(dataset,
            x_vars=['Поверхностная плотность, г/м2', 'Прочность при растяжении, МПа',
                    'Потребление смолы, г/м2', 'Угол нашивки',
                    'Шаг нашивки', 'Плотность нашивки'],
             y_vars=['Модуль упругости при растяжении, ГПа'],
             palette='bright')

In [ ]:
#попарные графики рассеяния точек
sns.set_style('darkgrid')
sns.pairplot(dataset,
            x_vars=['Соотношение матрица-наполнитель', 'Плотность, кг/м3',
                    'Модуль упругости, ГПа', 'Количество отвердителя, м.%',
                    'Содержание эпоксидных групп,%_2', 'Температура вспышки, С_2'],
             y_vars=['Прочность при растяжении, МПа'],
             palette='bright')

In [ ]:
#попарные графики рассеяния точек
sns.set_style('darkgrid')
sns.pairplot(dataset,
            x_vars=['Поверхностная плотность, г/м2', 'Модуль упругости при растяжении, ГПа',
                    'Потребление смолы, г/м2', 'Угол нашивки',
                    'Шаг нашивки', 'Плотность нашивки'],
             y_vars=['Прочность при растяжении, МПа'],
             palette='bright')

In [ ]:
#изменение формы матрицы на треугольную
matrix = np.triu(dataset.corr())
#создание полотна для тепловой карты
x = plt.subplots(figsize = (11, 11))
#строим тепловую кату
sns.heatmap(dataset.corr(), annot = True, cmap= 'coolwarm', mask=matrix)

In [ ]:
fig, ax = plt.subplots(figsize = (12, 6))
dataset.plot(kind = 'kde', ax = ax)
#убеждаемся, что данные находятся в разных диапазонах - нужна нормализаия

**Нормализуем данные**

In [ ]:
#нормализуем данные с помощью MinMaxScaler() 
minmax_scaler = MinMaxScaler()
dataset_norm = pd.DataFrame(minmax_scaler.fit_transform(np.array(dataset)), columns = dataset.columns)
dataset_norm.describe()

In [ ]:
dataset_norm

In [ ]:
plt.figure(figsize = (35,10))
ax = sns.boxplot(data = dataset_norm)
ax.set_xticklabels(ax.get_xticklabels(),rotation=30)

In [ ]:
#построим гистограммы распределения каждой из переменных
dataset_norm.hist(figsize = (20,20), color = 'b')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize = (20, 8))
dataset_norm.plot(kind = 'kde', ax = ax)

In [ ]:
#попарные графики рассеяния точек
sns.set_style('darkgrid')
sns.pairplot(dataset_norm,
            x_vars=['Соотношение матрица-наполнитель', 'Плотность, кг/м3',
                    'Модуль упругости, ГПа', 'Количество отвердителя, м.%',
                    'Содержание эпоксидных групп,%_2', 'Температура вспышки, С_2'],
             y_vars=['Модуль упругости при растяжении, ГПа'],
             palette='bright')

In [ ]:
#попарные графики рассеяния точек
sns.set_style('darkgrid')
sns.pairplot(dataset_norm,
            x_vars=['Поверхностная плотность, г/м2', 'Прочность при растяжении, МПа',
                    'Потребление смолы, г/м2', 'Угол нашивки',
                    'Шаг нашивки', 'Плотность нашивки'],
             y_vars=['Модуль упругости при растяжении, ГПа'],
             palette='bright')

In [ ]:
#попарные графики рассеяния точек
sns.set_style('darkgrid')
sns.pairplot(dataset_norm,
            x_vars=['Соотношение матрица-наполнитель', 'Плотность, кг/м3',
                    'Модуль упругости, ГПа', 'Количество отвердителя, м.%',
                    'Содержание эпоксидных групп,%_2', 'Температура вспышки, С_2'],
             y_vars=['Прочность при растяжении, МПа'],
             palette='bright')

In [ ]:
#попарные графики рассеяния точек
sns.set_style('darkgrid')
sns.pairplot(dataset_norm,
            x_vars=['Поверхностная плотность, г/м2', 'Модуль упругости при растяжении, ГПа',
                    'Потребление смолы, г/м2', 'Угол нашивки',
                    'Шаг нашивки', 'Плотность нашивки'],
             y_vars=['Прочность при растяжении, МПа'],
             palette='bright')

In [ ]:
dataset_norm.corr()

In [ ]:
#изменение формы матрицы на треугольную
matrix = np.triu(dataset_norm.corr())
#создание полотна для тепловой карты
x = plt.subplots(figsize = (11, 11))
#строим тепловую кату
sns.heatmap(dataset_norm.corr(), annot = True, cmap= 'coolwarm', mask=matrix)

In [ ]:
normalizer = Normalizer()
dataset_norm_n = pd.DataFrame(normalizer.fit_transform(np.array(dataset)), columns = dataset.columns)
dataset_norm_n

In [ ]:
dataset_norm_n.describe()

In [ ]:
dataset_norm_n.corr()

In [ ]:
#попарные графики рассеяния точек
sns.set_style('darkgrid')
sns.pairplot(dataset_norm_n,
            x_vars=['Соотношение матрица-наполнитель', 'Плотность, кг/м3',
                    'Модуль упругости, ГПа', 'Количество отвердителя, м.%',
                    'Содержание эпоксидных групп,%_2', 'Температура вспышки, С_2'],
             y_vars=['Модуль упругости при растяжении, ГПа'],
             kind='scatter', palette='bright')

In [ ]:
#попарные графики рассеяния точек
sns.set_style('darkgrid')
sns.pairplot(dataset_norm_n,
            x_vars=['Поверхностная плотность, г/м2', 'Прочность при растяжении, МПа',
                    'Потребление смолы, г/м2', 'Угол нашивки',
                    'Шаг нашивки', 'Плотность нашивки'],
             y_vars=['Модуль упругости при растяжении, ГПа'],
             kind='scatter', palette='bright')

In [ ]:
#попарные графики рассеяния точек
sns.set_style('darkgrid')
sns.pairplot(dataset_norm_n,
            x_vars=['Соотношение матрица-наполнитель', 'Плотность, кг/м3',
                    'Модуль упругости, ГПа', 'Количество отвердителя, м.%',
                    'Содержание эпоксидных групп,%_2', 'Температура вспышки, С_2'],
             y_vars=['Прочность при растяжении, МПа'],
             kind='scatter', palette='bright')

In [ ]:
#попарные графики рассеяния точек
sns.set_style('darkgrid')
sns.pairplot(dataset_norm_n,
            x_vars=['Поверхностная плотность, г/м2', 'Модуль упругости при растяжении, ГПа',
                    'Потребление смолы, г/м2', 'Угол нашивки',
                    'Шаг нашивки', 'Плотность нашивки'],
             y_vars=['Прочность при растяжении, МПа'],
             kind='scatter', palette='bright')

In [ ]:
#изменение формы матрицы на треугольную
matrix = np.triu(dataset_norm_n.corr())
#создание полотна для тепловой карты
x = plt.subplots(figsize = (11, 11))
#строим тепловую кату
sns.heatmap(dataset_norm_n.corr(), annot = True, cmap= 'coolwarm', mask=matrix)

**Стандартизуем данные**

In [ ]:
std_scaler = StandardScaler()
dataset_std = pd.DataFrame(std_scaler.fit_transform(np.array(dataset)), columns = dataset.columns)
dataset_std.describe()

In [ ]:
dataset_std

In [ ]:
plt.figure(figsize = (16,10))
ax = sns.boxplot(data = dataset_std)
ax.set_xticklabels(ax.get_xticklabels(),rotation=30)

In [ ]:
fig, ax = plt.subplots(figsize = (20, 8))
dataset_std.plot(kind = 'kde', ax = ax)

In [ ]:
#попарные графики рассеяния точек
sns.set_style('darkgrid')
sns.pairplot(dataset_std,
            x_vars=['Соотношение матрица-наполнитель', 'Плотность, кг/м3',
                    'Модуль упругости, ГПа', 'Количество отвердителя, м.%',
                    'Содержание эпоксидных групп,%_2', 'Температура вспышки, С_2'],
             y_vars=['Модуль упругости при растяжении, ГПа'],
             palette='bright')

In [ ]:
#попарные графики рассеяния точек
sns.set_style('darkgrid')
sns.pairplot(dataset_std,
            x_vars=['Поверхностная плотность, г/м2', 'Прочность при растяжении, МПа',
                    'Потребление смолы, г/м2', 'Угол нашивки',
                    'Шаг нашивки', 'Плотность нашивки'],
             y_vars=['Модуль упругости при растяжении, ГПа'],
             palette='bright')

In [ ]:
#попарные графики рассеяния точек
sns.set_style('darkgrid')
sns.pairplot(dataset_std,
            x_vars=['Соотношение матрица-наполнитель', 'Плотность, кг/м3',
                    'Модуль упругости, ГПа', 'Количество отвердителя, м.%',
                    'Содержание эпоксидных групп,%_2', 'Температура вспышки, С_2'],
             y_vars=['Прочность при растяжении, МПа'],
             palette='bright')

In [ ]:
#попарные графики рассеяния точек
sns.set_style('darkgrid')
sns.pairplot(dataset_std,
            x_vars=['Поверхностная плотность, г/м2', 'Модуль упругости при растяжении, ГПа',
                    'Потребление смолы, г/м2', 'Угол нашивки',
                    'Шаг нашивки', 'Плотность нашивки'],
             y_vars=['Прочность при растяжении, МПа'],
             palette='bright')

In [ ]:
dataset_std.corr()

In [ ]:
#изменение формы матрицы на треугольную
matrix = np.triu(dataset_std.corr())
#создание полотна для тепловой карты
x = plt.subplots(figsize = (11, 11))
#строим тепловую кату
sns.heatmap(dataset_std.corr(), annot = True, cmap= 'coolwarm', mask=matrix)

In [ ]:
#создадим новый файл
dataset_norm_n.to_excel('dataset2.xlsx', index = False)
print('Файл успешно сохранился!')

**Создадим и обучим модели**

In [ ]:
#импорт
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error, accuracy_score, precision_score
from sklearn.metrics import mean_absolute_error, accuracy_score, precision_score
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

from tensorflow import keras as keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Activation
from pandas import read_excel, DataFrame, Series
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.models import Sequential
from numpy.random import seed
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

**Модели для переменной "Модуль упругости при растяжении" (upr)**

In [ ]:
dataset_upr = dataset_norm_n
dataset_upr

In [ ]:
dataset_upr.drop(dataset_upr.columns[8], axis = 1, inplace=True)
dataset_upr

In [ ]:
dataset_upr.info()

In [ ]:
#разбиваем на выборки
X_train_upr, X_test_upr, y_train_upr, y_test_upr = train_test_split(
    dataset_upr.loc[:, dataset_upr.columns != 'Модуль упругости при растяжении, ГПа'],
    dataset_upr[['Модуль упругости при растяжении, ГПа']],
       test_size = 0.3,
    random_state = 42)

In [ ]:
X_train_upr.shape

In [ ]:
X_test_upr.shape

In [ ]:
y_train_upr.shape

In [ ]:
y_test_upr.shape

**Случайный лес**

In [ ]:
#построение модели и вузуализация метода случайный лес
rfr = RandomForestRegressor(n_estimators=15, max_depth=7, random_state=33)
rfr.fit(X_train_upr, y_train_upr.values)
y_forest = rfr.predict(X_test_upr)


In [ ]:
mean_absolute_forest = mean_absolute_error(y_test_upr, y_forest)*100
mean_squared_forest = mean_squared_error(y_test_upr, y_forest)
r2_score_forest = r2_score(y_test_upr, y_forest)
mean_absolute_percentage_forest = mean_absolute_percentage_error(y_test_upr, y_forest)*100
print('Средняя абсолютная ошибка', mean_absolute_forest)
print('Средняя квадратичная ошибка', mean_squared_forest)
print('Коэффициент детерминации', r2_score_forest)
print('Средняя абсолютная процентная ошибка', mean_absolute_percentage_forest)

In [ ]:
plt.figure(figsize = (10, 7))
plt.title('Тестовые и прогнозные значения Метода случайного леса')
plt.plot(y_forest, label = 'Прогноз', color = 'red')
plt.plot(y_test_upr.values, label = 'Тест', color = 'green')
plt.xlabel('Количество наблюдений')
plt.ylabel('Модуль упругости при растяжении, ГПа')
plt.legend()
plt.grid(True)

**К-ближайших соседей**

In [ ]:
#построение модели и вузуализация Метод К ближайших соседей
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train_upr, y_train_upr)
y_knn = knn.predict(X_test_upr)

In [ ]:
mean_absolute_knn = mean_absolute_error(y_test_upr, y_knn)*100
mean_squared_knn = mean_squared_error(y_test_upr, y_knn)
r2_score_knn = r2_score(y_test_upr, y_knn)
mean_absolute_percentage_knn = mean_absolute_percentage_error(y_test_upr, y_knn)*100
print('Средняя абсолютная ошибка', mean_absolute_knn)
print('Средняя квадратичная ошибка', mean_squared_knn)
print('Коэффициент детерминации', r2_score_knn)
print('Средняя абсолютная процентная ошибка', mean_absolute_percentage_knn)

In [ ]:
plt.figure(figsize = (10, 7))
plt.title('Тестовые и прогнозные значения Метода К-ближайших соседей')
plt.plot(y_knn, label = 'Прогноз', color = 'red')
plt.plot(y_test_upr.values, label = 'Тест', color = 'green')
plt.xlabel('Количество наблюдений')
plt.ylabel('Модуль упругости при растяжении, ГПа')
plt.legend()
plt.grid(True)

**Дерево решений**

In [ ]:
#построение модели и вузуализация Метода дерево решений
dtr = DecisionTreeRegressor()
dtr.fit(X_train_upr, y_train_upr.values)
y_dtr = dtr.predict(X_test_upr)

In [ ]:
mean_absolute_dtr = mean_absolute_error(y_test_upr, y_dtr)*100
mean_squared_dtr = mean_squared_error(y_test_upr, y_dtr)
r2_score_dtr = r2_score(y_test_upr, y_dtr)
mean_absolute_percentage_dtr = mean_absolute_percentage_error(y_test_upr, y_dtr)*100
print('Средняя абсолютная ошибка', mean_absolute_dtr)
print('Средняя квадратичная ошибка', mean_squared_dtr)
print('Коэффициент детерминации', r2_score_dtr)
print('Средняя абсолютная процентная ошибка', mean_absolute_percentage_dtr)

In [ ]:
plt.figure(figsize = (10, 7))
plt.title('Тестовые и прогнозные значения Метода дерева решений')
plt.plot(y_dtr, label = 'Прогноз', color = 'red')
plt.plot(y_test_upr.values, label = 'Тест', color = 'green')
plt.xlabel('Количество наблюдений')
plt.ylabel('Модуль упругости при растяжении, ГПа')
plt.legend()
plt.grid(True)

In [ ]:
res = {'Модель':['Случайный лес',
                 'К-ближайших соседей', 'Дерево решений'],
       'Средняя абсолютная ошибка' : [mean_absolute_forest,
                                      mean_absolute_knn, mean_absolute_dtr],
       'Средняя квадратичная ошибка' : [mean_squared_forest,
                                      mean_squared_knn, mean_squared_dtr],
       'Коэффициент детерминации' : [r2_score_forest,
                                      r2_score_knn, r2_score_dtr],
       'Средняя абсолютная процентная ошибка': [mean_absolute_percentage_forest,
                                      mean_absolute_percentage_knn, mean_absolute_percentage_dtr]}

In [ ]:
res

In [ ]:
res = pd.DataFrame(res)
res

**Подбор гиперпараметров**

In [ ]:
#Случайный лес
parametrs = { 'n_estimators': [200, 300],
              'max_depth': [9, 15],
              'max_features': ['auto'],
              'criterion': ['mse'] }
grid = GridSearchCV(estimator = rfr, param_grid = parametrs, cv = 10)
grid.fit(X_train_upr, y_train_upr)

In [ ]:
#параметры лучшей модели
grid.best_params_

In [ ]:
#ошибка на лучшей модели
print(grid.best_score_)

In [ ]:
rfr_grid = RandomForestRegressor(n_estimators=200, criterion='mse', max_depth=9, max_features = 'auto')

In [ ]:
rfr_grid.fit(X_train_upr, y_train_upr.values)

In [ ]:
y_forest_grid = rfr_grid.predict(X_test_upr)

In [ ]:
mean_absolute_forest1 = mean_absolute_error(y_test_upr, y_forest_grid)*100
mean_squared_forest1 = mean_squared_error(y_test_upr, y_forest_grid)
r2_score_forest1 = r2_score(y_test_upr, y_forest_grid)
mean_absolute_percentage_forest1 = mean_absolute_percentage_error(y_test_upr, y_forest_grid)*100
print('Средняя абсолютная ошибка', mean_absolute_forest1)
print('Средняя квадратичная ошибка', mean_squared_forest1)
print('Коэффициент детерминации', r2_score_forest1)
print('Средняя абсолютная процентная ошибка', mean_absolute_percentage_forest1)

In [ ]:
plt.figure(figsize = (10, 7))
plt.title('Тестовые и прогнозные значения Метода случайного леса')
plt.plot(y_forest, label = 'Прогноз', color = 'red')
plt.plot(y_test_upr.values, label = 'Тест', color = 'green')
plt.xlabel('Количество наблюдений')
plt.ylabel('Модуль упругости при растяжении, ГПа')
plt.legend()
plt.grid(True)

In [ ]:
#К ближайших соседей
knn = KNeighborsRegressor()
knn_params = {'n_neighbors' : range(1, 301, 2), 
          'weights' : ['uniform', 'distance'],
          'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute']
          }
gs = GridSearchCV(knn, knn_params, cv = 10, verbose = 1, n_jobs =-1, scoring = 'r2')
gs.fit(X_train_upr, y_train_upr)

In [ ]:
#параметры лучшей модели
gs.best_params_

In [ ]:
#ошибка на лучшей модели
print(gs.best_score_)

In [ ]:
knn_grid = KNeighborsRegressor(algorithm = 'brute', n_neighbors = 13, weights = 'distance')

In [ ]:
knn_grid.fit(X_train_upr, y_train_upr)

In [ ]:
y_knn_grid = rfr_grid.predict(X_test_upr)

In [ ]:
mean_absolute_knn1 = mean_absolute_error(y_test_upr, y_knn_grid)*100
mean_squared_knn1 = mean_squared_error(y_test_upr, y_knn_grid)
r2_score_knn1 = r2_score(y_test_upr, y_knn_grid)
mean_absolute_percentage_knn1 = mean_absolute_percentage_error(y_test_upr, y_knn_grid)*100
print('Средняя абсолютная ошибка', mean_absolute_knn1)
print('Средняя квадратичная ошибка', mean_squared_knn1)
print('Коэффициент детерминации', r2_score_knn1)
print('Средняя абсолютная процентная ошибка', mean_absolute_percentage_knn1)

In [ ]:
plt.figure(figsize = (10, 7))
plt.title('Тестовые и прогнозные значения Метода случайного леса')
plt.plot(y_knn_grid, label = 'Прогноз', color = 'red')
plt.plot(y_test_upr.values, label = 'Тест', color = 'green')
plt.xlabel('Количество наблюдений')
plt.ylabel('Модуль упругости при растяжении, ГПа')
plt.legend()
plt.grid(True)

In [ ]:
#Дерево решений
param_gr = {'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
               'splitter': ['best', 'random'],
               'max_depth': [3,5,7,9,11],
               'min_samples_split': [200,250,300],
               'min_samples_leaf': [100,150,200],
               'max_features': ['auto', 'sqrt', 'log2']}
grids = GridSearchCV(dtr, param_gr, cv = 10, verbose = 1, n_jobs=-1, scoring = 'r2')
grids.fit(X_train_upr, y_train_upr)

In [ ]:
#параметры лучшей модели
grids.best_params_

In [ ]:
#ошибка на лучшей модели
print(grids.best_score_)

In [ ]:
dtr_grid = DecisionTreeRegressor(criterion='poisson', max_depth=5, max_features='auto',
                      min_samples_leaf=100, min_samples_split=200, splitter='best')

In [ ]:
dtr_grid.fit(X_train_upr, y_train_upr)

In [ ]:
y_dtr_grid = rfr_grid.predict(X_test_upr)

In [ ]:
mean_absolute_dtr1 = mean_absolute_error(y_test_upr, y_dtr_grid)*100
mean_squared_dtr1 = mean_squared_error(y_test_upr, y_dtr_grid)
r2_score_dtr1 = r2_score(y_test_upr, y_dtr_grid)
mean_absolute_percentage_dtr1 = mean_absolute_percentage_error(y_test_upr, y_dtr_grid)*100
print('Средняя абсолютная ошибка', mean_absolute_dtr1)
print('Средняя квадратичная ошибка', mean_squared_dtr1)
print('Коэффициент детерминации', r2_score_dtr1)
print('Средняя абсолютная процентная ошибка', mean_absolute_percentage_dtr1)

In [ ]:
plt.figure(figsize = (10, 7))
plt.title('Тестовые и прогнозные значения Метода случайного леса')
plt.plot(y_dtr_grid, label = 'Прогноз', color = 'red')
plt.plot(y_test_upr.values, label = 'Тест', color = 'green')
plt.xlabel('Количество наблюдений')
plt.ylabel('Модуль упругости при растяжении, ГПа')
plt.legend()
plt.grid(True)

In [ ]:
res1 = {'Модель':['Случайный лес',
                 'К-ближайших соседей', 'Дерево решений'],
       'Средняя абсолютная ошибка' : [mean_absolute_forest1,
                                      mean_absolute_knn1, mean_absolute_dtr1],
       'Средняя квадратичная ошибка' : [mean_squared_forest1,
                                      mean_squared_knn1, mean_squared_dtr1],
       'Коэффициент детерминации' : [r2_score_forest1,
                                      r2_score_knn1, r2_score_dtr1],
       'Средняя абсолютная процентная ошибка': [mean_absolute_percentage_forest1,
                                      mean_absolute_percentage_knn1, mean_absolute_percentage_dtr1]}

In [ ]:
res1

In [ ]:
res1 = pd.DataFrame(res1)
res1